In [ ]:
## Libraries allowed
# pandas
# numpy

# scipy
# xgboost, lightGBM, Catboost
# sklearn
# category_encoders
# feature_engine.outliers
# glmnet_python



In [29]:
import sys
import time
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from math import sqrt
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingRegressor
import random



import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import matplotlib.pyplot as plt
import seaborn as sns


In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.tree import plot_tree, DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, KFold


In [26]:
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 2)

In [98]:
import time 
from functools import wraps
  
  
def timer_func(func): 
    # This function shows the execution time of  
    # the function object passed 
    @wraps(func)
    def wrap_func(*args, **kwargs): 
        t1 = time.time() 
        result = func(*args, **kwargs) 
        t2 = time.time() 
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s') 
        return result 
    return wrap_func 
  
  


In [99]:
import time 
#import time

@timer_func
def tmp_1():
    time.sleep(1)
    print("Hi")

print(tmp_1.__name__)

tmp_1()


@timer_func
def long_time(n): 
    for i in range(n): 
        for j in range(100000): 
            i*j 
  
  
long_time(5) 

tmp_1
Hi
Function 'tmp_1' executed in 1.0055s
Function 'long_time' executed in 0.0351s


In [95]:
import time

from tqdm import tqdm
for i in tqdm(range(1)):
    time.sleep(i)
    print(i)
    pass



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 7096.96it/s]

0


In [122]:
test_file = "test.csv"
test_y_file = "test_y.csv"
train_file = "train.csv"

folder = "proj1/fold1/"

train_df = pd.read_csv(folder + train_file)
test_df = pd.read_csv(folder + test_file)
test_y_df = pd.read_csv(folder + test_y_file)


In [123]:
print("train_data.shape=", train_df.shape)

train_df.head()

train_data.shape= (2051, 83)


,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,Land_Slope,Neighborhood,Condition_1,Condition_2,Bldg_Type,House_Style,Overall_Qual,Overall_Cond,Year_Built,Year_Remod_Add,Roof_Style,Roof_Matl,Exterior_1st,Exterior_2nd,Mas_Vnr_Type,Mas_Vnr_Area,Exter_Qual,Exter_Cond,Foundation,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_SF_1,BsmtFin_Type_2,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,Heating,Heating_QC,Central_Air,Electrical,First_Flr_SF,Second_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,Kitchen_Qual,TotRms_AbvGrd,Functional,Fireplaces,Fireplace_Qu,Garage_Type,Garage_Yr_Blt,Garage_Finish,Garage_Cars,Garage_Area,Garage_Qual,Garage_Cond,Paved_Drive,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Pool_Area,Pool_QC,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Longitude,Latitude,Sale_Price
0,526350040,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,North_Ames,Feedr,Norm,OneFam,One_Story,Average,Above_Average,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0,Typical,Typical,CBlock,Typical,Typical,No,Rec,6,LwQ,144,270,882,GasA,Typical,Y,SBrkr,896,0,0,896,0,0,1,0,2,1,Typical,5,Typ,0,No_Fireplace,Attchd,1961.0,Unf,1,730,Typical,Typical,Paved,140,0,0,0,120,0,No_Pool,Minimum_Privacy,None,0,6,2010,WD,Normal,-93.62,42.05,105000
1,526351010,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,Gtl,North_Ames,Norm,Norm,OneFam,One_Story,Above_Average,Above_Average,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108,Typical,Typical,CBlock,Typical,Typical,No,ALQ,1,Unf,0,406,1329,GasA,Typical,Y,SBrkr,1329,0,0,1329,0,0,1,1,3,1,Good,6,Typ,0,No_Fireplace,Attchd,1958.0,Unf,1,312,Typical,Typical,Paved,393,36,0,0,0,0,No_Pool,No_Fence,Gar2,12500,6,2010,WD,Normal,-93.62,42.05,172000
2,527105010,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,OneFam,Two_Story,Average,Average,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0,Typical,Typical,PConc,Good,Typical,No,GLQ,3,Unf,0,137,928,GasA,Good,Y,SBrkr,928,701,0,1629,0,0,2,1,3,1,Typical,6,Typ,1,Typical,Attchd,1997.0,Fin,2,482,Typical,Typical,Paved,212,34,0,0,0,0,No_Pool,Minimum_Privacy,None,0,3,2010,WD,Normal,-93.64,42.06,189900
3,527105030,Two_Story_1946_and_Newer,Residential_Low_Density,78,9978,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,OneFam,Two_Story,Above_Average,Above_Average,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20,Typical,Typical,PConc,Typical,Typical,No,GLQ,3,Unf,0,324,926,GasA,Excellent,Y,SBrkr,926,678,0,1604,0,0,2,1,3,1,Good,7,Typ,1,Good,Attchd,1998.0,Fin,2,470,Typical,Typical,Paved,360,36,0,0,0,0,No_Pool,No_Fence,None,0,6,2010,WD,Normal,-93.64,42.06,195500
4,527127150,One_Story_PUD_1946_and_Newer,Residential_Low_Density,41,4920,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Stone_Brook,Norm,Norm,TwnhsE,One_Story,Very_Good,Average,2001,2001,Gable,CompShg,CemntBd,CmentBd,None,0,Good,Typical,PConc,Good,Typical,Mn,GLQ,3,Unf,0,722,1338,GasA,Excellent,Y,SBrkr,1338,0,0,1338,1,0,2,0,2,1,Good,6,Typ,0,No_Fireplace,Attchd,2001.0,Fin,2,582,Typical,Typical,Paved,0,0,170,0,0,0,No_Pool,No_Fence,None,0,4,2010,WD,Normal,-93.63,42.06,213500


In [124]:
train_df.describe()

,PID,Lot_Frontage,Lot_Area,Year_Built,Year_Remod_Add,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,TotRms_AbvGrd,Fireplaces,Garage_Yr_Blt,Garage_Cars,Garage_Area,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Pool_Area,Misc_Val,Mo_Sold,Year_Sold,Longitude,Latitude,Sale_Price
count,2.05e+03,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,1943.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00,2051.00
mean,7.11e+08,57.42,9990.19,1971.62,1984.34,104.20,4.16,50.16,546.04,1042.86,1151.84,331.74,4.57,1488.16,0.44,0.06,1.55,0.37,2.84,1.04,6.41,0.60,1978.26,1.77,473.76,93.84,46.61,22.28,2.55,15.79,1.77,37.92,6.24,2007.80,-93.64,42.03,180784.05
std,1.89e+08,32.79,7645.95,30.13,20.77,183.02,2.22,170.11,432.76,431.30,384.40,427.79,43.28,501.75,0.53,0.24,0.55,0.50,0.83,0.21,1.57,0.64,25.78,0.76,215.46,121.66,66.66,62.88,25.60,55.15,31.89,361.12,2.71,1.32,0.03,0.02,82566.70
min,5.26e+08,0.00,1300.00,1872.00,1950.00,0.00,0.00,0.00,0.00,0.00,334.00,0.00,0.00,334.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,1895.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2006.00,-93.69,41.99,12789.00
25%,5.28e+08,43.00,7345.00,1954.00,1966.00,0.00,3.00,0.00,216.00,780.50,864.00,0.00,0.00,1115.00,0.00,0.00,1.00,0.00,2.00,1.00,5.00,0.00,1960.00,1.00,320.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,2007.00,-93.66,42.02,128500.00
50%,5.35e+08,63.00,9360.00,1974.00,1993.00,0.00,3.00,0.00,451.00,980.00,1077.00,0.00,0.00,1431.00,0.00,0.00,2.00,0.00,3.00,1.00,6.00,1.00,1980.00,2.00,480.00,0.00,26.00,0.00,0.00,0.00,0.00,0.00,6.00,2008.00,-93.64,42.03,160000.00
75%,9.07e+08,78.00,11422.50,2001.00,2004.00,164.50,7.00,0.00,778.50,1296.00,1383.00,697.50,0.00,1729.50,1.00,0.00,2.00,1.00,3.00,1.00,7.00,1.00,2002.00,2.00,576.00,168.00,69.00,0.00,0.00,0.00,0.00,0.00,8.00,2009.00,-93.62,42.05,213066.50
max,9.24e+08,313.00,215245.00,2010.00,2010.00,1600.00,7.00,1526.00,2336.00,3206.00,3820.00,2065.00,697.00,4676.00,3.00,2.00,4.00,2.00,8.00,3.00,14.00,3.00,2207.00,5.00,1488.00,870.00,742.00,1012.00,508.00,490.00,738.00,12500.00,12.00,2010.00,-93.59,42.06,755000.00


In [125]:
train_df.describe(include=object)

,MS_SubClass,MS_Zoning,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,Land_Slope,Neighborhood,Condition_1,Condition_2,Bldg_Type,House_Style,Overall_Qual,Overall_Cond,Roof_Style,Roof_Matl,Exterior_1st,Exterior_2nd,Mas_Vnr_Type,Exter_Qual,Exter_Cond,Foundation,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_Type_2,Heating,Heating_QC,Central_Air,Electrical,Kitchen_Qual,Functional,Fireplace_Qu,Garage_Type,Garage_Finish,Garage_Qual,Garage_Cond,Paved_Drive,Pool_QC,Fence,Misc_Feature,Sale_Type,Sale_Condition
count,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051
unique,16,7,2,3,4,4,2,5,3,28,9,7,5,8,10,9,6,6,16,16,4,4,5,6,6,6,5,7,7,6,5,2,6,5,8,6,7,4,6,6,3,5,5,5,10,6
top,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,North_Ames,Norm,Norm,OneFam,One_Story,Average,Average,Gable,CompShg,VinylSd,VinylSd,None,Typical,Typical,PConc,Typical,Typical,No,GLQ,Unf,GasA,Excellent,Y,SBrkr,Typical,Typ,No_Fireplace,Attchd,Unf,Typical,Typical,Paved,No_Pool,No_Fence,None,WD,Normal
freq,757,1577,2040,1909,1334,1854,2049,1515,1950,320,1764,2025,1692,1031,583,1156,1616,2021,722,716,1233,1256,1785,926,886,1831,1324,618,1748,2021,1039,1911,1878,1043,1913,981,1215,854,1822,1866,1860,2044,1651,1975,1781,1673


In [126]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 83 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PID                 2051 non-null   int64  
 1   MS_SubClass         2051 non-null   object 
 2   MS_Zoning           2051 non-null   object 
 3   Lot_Frontage        2051 non-null   int64  
 4   Lot_Area            2051 non-null   int64  
 5   Street              2051 non-null   object 
 6   Alley               2051 non-null   object 
 7   Lot_Shape           2051 non-null   object 
 8   Land_Contour        2051 non-null   object 
 9   Utilities           2051 non-null   object 
 10  Lot_Config          2051 non-null   object 
 11  Land_Slope          2051 non-null   object 
 12  Neighborhood        2051 non-null   object 
 13  Condition_1         2051 non-null   object 
 14  Condition_2         2051 non-null   object 
 15  Bldg_Type           2051 non-null   object 
 16  House_

In [159]:
train_df.select_dtypes(include=['number']).head()

train_df.select_dtypes(include=['number']).columns.to_list()


train_df.columns.shape

train_df.select_dtypes(include=['number'])#.columns.to_list()

train_df.select_dtypes(include=['object']).columns.shape

train_df.select_dtypes(include=['object']).head()

train_df.select_dtypes(include=['object']).isnull().any()

train_df.select_dtypes(include=['object']).columns

Index(['MS_SubClass', 'MS_Zoning', 'Street', 'Alley', 'Lot_Shape',
       'Land_Contour', 'Utilities', 'Lot_Config', 'Land_Slope', 'Neighborhood',
       'Condition_1', 'Condition_2', 'Bldg_Type', 'House_Style',
       'Overall_Qual', 'Overall_Cond', 'Roof_Style', 'Roof_Matl',
       'Exterior_1st', 'Exterior_2nd', 'Mas_Vnr_Type', 'Exter_Qual',
       'Exter_Cond', 'Foundation', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure',
       'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Heating', 'Heating_QC',
       'Central_Air', 'Electrical', 'Kitchen_Qual', 'Functional',
       'Fireplace_Qu', 'Garage_Type', 'Garage_Finish', 'Garage_Qual',
       'Garage_Cond', 'Paved_Drive', 'Pool_QC', 'Fence', 'Misc_Feature',
       'Sale_Type', 'Sale_Condition'],
      dtype='object')

In [164]:


train_df[['PID', 'Lot_Frontage', 'Lot_Area', 'Year_Built', 'Year_Remod_Add',
       'Mas_Vnr_Area', 'BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF',
       'Total_Bsmt_SF', 'First_Flr_SF', 'Second_Flr_SF', 'Low_Qual_Fin_SF',
       'Gr_Liv_Area', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Full_Bath',
       'Half_Bath', 'Bedroom_AbvGr', 'Kitchen_AbvGr', 'TotRms_AbvGrd',
       'Fireplaces', 'Garage_Yr_Blt', 'Garage_Cars', 'Garage_Area',
       'Wood_Deck_SF', 'Open_Porch_SF', 'Enclosed_Porch', 'Three_season_porch',
       'Screen_Porch', 'Pool_Area', 'Misc_Val', 'Mo_Sold', 'Year_Sold',
       'Longitude', 'Latitude', 'Sale_Price']]

train_df.select_dtypes(include=['number']).columns.tolist()

['PID',
 'Lot_Frontage',
 'Lot_Area',
 'Year_Built',
 'Year_Remod_Add',
 'Mas_Vnr_Area',
 'BsmtFin_SF_1',
 'BsmtFin_SF_2',
 'Bsmt_Unf_SF',
 'Total_Bsmt_SF',
 'First_Flr_SF',
 'Second_Flr_SF',
 'Low_Qual_Fin_SF',
 'Gr_Liv_Area',
 'Bsmt_Full_Bath',
 'Bsmt_Half_Bath',
 'Full_Bath',
 'Half_Bath',
 'Bedroom_AbvGr',
 'Kitchen_AbvGr',
 'TotRms_AbvGrd',
 'Fireplaces',
 'Garage_Yr_Blt',
 'Garage_Cars',
 'Garage_Area',
 'Wood_Deck_SF',
 'Open_Porch_SF',
 'Enclosed_Porch',
 'Three_season_porch',
 'Screen_Porch',
 'Pool_Area',
 'Misc_Val',
 'Mo_Sold',
 'Year_Sold',
 'Longitude',
 'Latitude',
 'Sale_Price']

In [130]:
def process_train_data(train_df):
    y = train_df['Sale_Price']
    y = np.log10(y)
    
    train_df = train_df.drop(columns=['PID','Longitude','Latitude','Sale_Price'], axis = 1)
    
    #df = df[column=['Lot_Frontage', 'Lot_Area']]
    train_df = train_df.select_dtypes(include=['number'])
    X = train_df.to_numpy()
    return (X, y)

In [131]:
(X_train, y_train) = process_train_data(train_df)

In [141]:
def process_test_data(test_df, test_y_df):
    y = test_y_df['Sale_Price']
    y = np.log10(y)
    
    test_df = test_df.drop(columns=['PID','Longitude','Latitude'], axis = 1)
    
    #df = df[column=['Lot_Frontage', 'Lot_Area']]
    test_df = test_df.select_dtypes(include=['number'])
    X = test_df.to_numpy()
    return (X, y)

In [148]:
(X_train, y_train) = process_train_data(train_df)
(X_test, y_test) = process_test_data(test_df, test_y_df)

In [151]:
#test_y_df['Sale_Price']
#test_df


#print(np.linspace(-10, 1, 100))
#print(np.logspace(-10, 1, 100))

print(np.linspace(1, 100, 100))
print(np.logspace(1, 100, 100))
#print(np.logspace(-10, 1, 100))


[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.
  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.
  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.
  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.
  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.
  99. 100.]
[1.e+001 1.e+002 1.e+003 1.e+004 1.e+005 1.e+006 1.e+007 1.e+008 1.e+009
 1.e+010 1.e+011 1.e+012 1.e+013 1.e+014 1.e+015 1.e+016 1.e+017 1.e+018
 1.e+019 1.e+020 1.e+021 1.e+022 1.e+023 1.e+024 1.e+025 1.e+026 1.e+027
 1.e+028 1.e+029 1.e+030 1.e+031 1.e+032 1.e+033 1.e+034 1.e+035 1.e+036
 1.e+037 1.e+038 1.e+039 1.e+040 1.e+041 1.e+042 1.e+043 1.e+044 1.e+045
 1.e+046 1.e+047 1.e+048 1.e+049 1.e+050 1.e+051 1.e+052 1.e+053 1.e+054
 1.e+055 1.e+056 1.e+057 1.e+058 1.e+059 1.e+060 1.e+

In [ ]:

ridgecv = RidgeCV(alphas = alphas, cv = 10, 
                  scoring = 'neg_mean_squared_error', 
                  normalize = True)
ridgecv.fit(X_train, Y_train)
ridgecv.alpha_


In [68]:
import numpy as np

dig = np.diag(np.array([1, 2, 0]))
dig = np.array([[1, 0, 1], 
                [0, 2, 0], 
                [3, 0, 3]])

print(np.linalg.svd(dig))
print(np.linalg.inv(dig))

print(np.round(dig @ np.linalg.inv(dig)))

print(np.linalg.matrix_rank(dig))
#np.linalg.inv(dig)

(U,S,Vh) = np.linalg.svd(dig)

print(Vh.T @ np.linalg.inv(np.diag(S)) @ U.T )

print(np.round(Vh.T @ Vh ))

print(np.round(U.T @ U ))

print(np.round(np.diag(S)))

#print(np.round(np.linalg.inv(dig) - (Vh.T @ np.linalg.inv(np.diag(S)) @ U.T )))

print(np.round(dig @ (Vh.T @ np.linalg.inv(np.diag(S)) @ U.T )))

SVDResult(U=array([[-0.31622777,  0.        ,  0.9486833 ],
       [ 0.        , -1.        ,  0.        ],
       [-0.9486833 ,  0.        , -0.31622777]]), S=array([4.47213595e+00, 2.00000000e+00, 3.77598896e-16]), Vh=array([[-0.70710678,  0.        , -0.70710678],
       [-0.        , -1.        ,  0.        ],
       [ 0.70710678,  0.        , -0.70710678]]))
[[-1.80143985e+16  0.00000000e+00  6.00479950e+15]
 [ 0.00000000e+00  5.00000000e-01  0.00000000e+00]
 [ 1.80143985e+16  0.00000000e+00 -6.00479950e+15]]
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
2
[[ 1.77654225e+15  0.00000000e+00 -5.92180750e+14]
 [ 0.00000000e+00  5.00000000e-01  0.00000000e+00]
 [-1.77654225e+15  0.00000000e+00  5.92180750e+14]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[4. 0. 0.]
 [0. 2. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
